## Import Required Packages

In [1]:
import torch
from torch.utils.data import DataLoader
import onnx
import pandas as pd
import numpy as np
from onnx_tf.backend import prepare
import tensorflow as tf

from pytorch_helper import DeepHEC,get_test_dataset
from tf_helper import predict

2023-06-08 10:25:12.789154: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 10:25:12.896158: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-08 10:25:12.896172: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-08 10:25:13.350623: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Model Configuration

In [2]:
# Epochs and batch size
MAX_EPOCH = 1000

# NN architecture
HIDDEN = 4
NEURONS = 150
INPUT = 6
OUTPUT = 256

# Learning rate schedule configuration
MAX_LR = 4e-2
START_LR = 0.001
END_LR = 2e-9
DIV_FACTOR = MAX_LR / START_LR
FINAL_DIV_FACTOR = MAX_LR / END_LR

# Regularization factor
REG_FACTOR = 0.00001

# DeepSHARQ delta
RI_RANGE = 0.1

## Prepare Path Variables

In [3]:
MODEL = "DeepSHARQ"

if MODEL == "DeepSHARQ":
    PATH = "./models/leakyReLU({},{})_K_range_{}_{}_max_lr={}_start_lr={}_final_lr={}_reg_factor={}/".format(HIDDEN,NEURONS,RI_RANGE,MAX_EPOCH,MAX_LR,START_LR,END_LR,REG_FACTOR)
elif MODEL == "DeepHEC":
    PATH = "./models/leakyReLU({},{})_fullmodel_{}_max_lr={}_div_factor={}_final_div_factor={}_reg_factor={}/".format(HIDDEN,NEURONS,MAX_EPOCH,MAX_LR,DIV_FACTOR,FINAL_DIV_FACTOR,REG_FACTOR)

PATH = "./models/"    

PYTORCH_PATH = PATH + "model"
ONNX_PATH = PATH + "model.onnx"
TF_PATH = PATH + "model.pb"
TFLITE_PATH = PATH + "model.tflite"
DATASET_PATH = './datasets/dataset.csv'

## Load Pytorch model

In [4]:
trained_model = DeepHEC(hidden_layers=HIDDEN, layer_size=NEURONS, inputs=INPUT, outputs=OUTPUT)
trained_model.load_state_dict(torch.load(PYTORCH_PATH))

<All keys matched successfully>

## Convert PyTorch to ONNX format

In [5]:
import onnx_tf
print(tf.__version__)

2.11.0


In [6]:
dummy_input = torch.randn(1,6)
torch.onnx.export(trained_model, dummy_input, ONNX_PATH)

onnx_model = onnx.load(ONNX_PATH)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(TF_PATH)

2023-06-08 10:25:14.226216: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-08 10:25:14.226248: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-08 10:25:14.226265: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pablo-pc): /proc/driver/nvidia/version does not exist
2023-06-08 10:25:14.226464: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Assets written to: ./models/model.pb/assets


INFO:tensorflow:Assets written to: ./models/model.pb/assets


## Convert ONNX to TFlite

In [7]:
# make a converter object from the saved tensorflow file
converter = tf.lite.TFLiteConverter.from_saved_model(TF_PATH)

# I had to explicitly state the ops
converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]

tf_lite_model = converter.convert()
# Save the model.
with open(TFLITE_PATH, 'wb') as f:
    f.write(tf_lite_model)

2023-06-08 10:25:15.243952: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-06-08 10:25:15.243988: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-06-08 10:25:15.244998: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./models/model.pb
2023-06-08 10:25:15.245927: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-08 10:25:15.245942: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./models/model.pb
2023-06-08 10:25:15.247873: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-06-08 10:25:15.248151: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-06-08 10:25:15.292403: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./models/model.pb
2023

## Test Converted Model Performance

In [8]:
# Load test dataset
test_dataset = get_test_dataset(DATASET_PATH)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
interpreter.allocate_tensors()
# Get input and outpt tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
# Check that the three models output the same result
with torch.no_grad():
    error_tf = 0
    error_tflite = 0
    total = 0
    for x, y in test_dataloader:
        # Predict with the three models
        out_py = trained_model(x.clone())
        out_tf = tf_rep.run(pd.DataFrame([pd.Series(x[0].tolist(), dtype="float32")]), verbose=0)[0][0]
        out_tflite = predict(interpreter, [np.float32(x) for x in x[0].tolist()], input_details, output_details)
        
        # Obtain model output
        out_py_argmax = out_py.argmax().item()
        out_tf_argmax = out_tf.argmax()
        out_tflite_argmax = out_tflite
        
        # Check for prediction erros in conversions
        
        if out_py_argmax != out_tf_argmax:
            print("Error TF")
            error_tf += 1
        elif out_py_argmax != out_tflite_argmax:
            print("Error TFLite")
            error_tflite += 1
        total += 1

print("Error TF {}, Error TFlite {}".format(error_tf/total, error_tflite/total))